In [7]:
DRIVER_PATH='C:/Users/Maathess/Desktop/Projet_annuel_flag/Datasets/chromedriver.exe'
search_term = ["ireland flag"]
number_images = 40
target_path = './Test'

In [8]:
from selenium import webdriver

In [9]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [10]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [11]:
def search_and_download(search_term:str,driver_path:str,target_path='./Test',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [12]:
import os 
import time
import requests
import io
from PIL import Image
import hashlib

for ele in search_term:
    search_and_download(search_term = ele, driver_path=DRIVER_PATH, target_path=target_path, number_images=number_images)

Found: 100 search results. Extracting links from 0:100
Found: 40 image links, done!
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Flag_of_Ireland.svg/2560px-Flag_of_Ireland.svg.png - as ./Test\ireland_flag\5ae3523afe.jpg
SUCCESS - saved https://ak.picdn.net/shutterstock/videos/19686439/thumb/1.jpg - as ./Test\ireland_flag\3bc6a9a805.jpg
SUCCESS - saved https://cdn11.bigcommerce.com/s-ey7tq/images/stencil/1280x1280/products/3320/18821/ireland-flag__72814.1575333970.jpg?c=2 - as ./Test\ireland_flag\7ca474da13.jpg
SUCCESS - saved https://i.pinimg.com/originals/f5/0b/c8/f50bc8f5f70d4891062da1050af5c149.png - as ./Test\ireland_flag\0d9f1298f4.jpg
SUCCESS - saved https://patchion.com/522/flag-of-ireland-small.jpg - as ./Test\ireland_flag\1bcabbe8c8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTdubWWBuorlvy1G6Oh_BAF-N2R0ttwtMlC6A&usqp=CAU - as ./Test\ireland_flag\4cb91578be.jpg
SUCCESS - saved https://www.vedi-express.com/34504-thickbox